In [1]:
%matplotlib inline
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from itertools import combinations 

from sklearn.cluster import KMeans

import folium

#Load data
data = pd.read_excel('E:/CoG/Data.xlsx', dtype = {'Location name':str, 'Location type':str})

# Color options
color_options = {'demand': 'red',
                 'supply': 'yellow',
                 'flow': 'black',
                 'cog': 'blue',
                 'candidate': 'black',
                 'other': 'gray'}

# Instantiate map
m = folium.Map(location=data[['Latitude','Longitude']].mean(),
               fit_bounds=[[data['Latitude'].min(),
                            data['Longitude'].min(),],
                           [data['Latitude'].max(),
                            data['Longitude'].max()]])

# Add volume points
for _, row in data.iterrows():
    folium.CircleMarker(location=[row['Latitude'],
                                 row['Longitude']],
                       radius=(row['Volume']**0.5),
                        color=color_options.get(str(row['Location type']).lower(),'gray'),
                        tooltip=str(row['Location name'])+' '+str(row['Volume'])).add_to(m)

# Zoom based on volume points
m.fit_bounds(data[['Latitude','Longitude']].values.tolist())

# Fit K-means for 1 centroid
kmeans = KMeans(n_clusters=1,random_state=0).fit(data.loc[data['Calc_Vol']>0, ['Latitude', 
                                                                  'Longitude']], 
                                    sample_weight=data.loc[data['Calc_Vol']>0, 
                                                           'Calc_Vol'])
# Get centers of gravity from K-means
cogs = kmeans.cluster_centers_
cogs = pd.DataFrame(cogs, columns=['Latitude',
                                   'Longitude'])

# Get volume assigned =
data['Cluster'] = kmeans.predict(data[['Latitude', 'Longitude']])
cogs = cogs.join(data.groupby('Cluster')['Volume'].sum())

# Include assigned COG coordinates in data by point 
data = data.join(cogs, on='Cluster', rsuffix='_COG')

# Add flow lines to centers of gravity to map
for _, row in data.iterrows():
    # Flow lines
    if str(row['Location type']).lower() in (['demand', 'supply']):
        folium.PolyLine([(row['Latitude'],
                          row['Longitude']),
                         (row['Latitude_COG'],
                          row['Longitude_COG'])],
                        color=color_options['flow'],
                        weight=(row['Volume']**0.5),
                        opacity=0.8).add_to(m)
                        
# Add centers of gravity to map
for _, row in cogs.iterrows():
    # New centers of gravity
    folium.CircleMarker(location=[row['Latitude'],
                                  row['Longitude']],
                        radius=(row['Volume']**0.5),
                        color=color_options['cog'],
                        tooltip=row['Volume']).add_to(m)

In [2]:
m

In [3]:
cogs

,Latitude,Longitude,Volume
0,13.543076,100.888386,1604
